In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
pd.reset_option('all')

In [ ]:
import os
os.chdir('/Users/ruimaciel/Desktop/Barcelona/Master_Thesis/ECB_Perceived_Cacophony')

current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)


In [ ]:
df_index=pd.read_csv('./Rui_final_notebooks/df_ready_for_index.csv')

In [ ]:
# Convert 'Date' column to datetime
df_index['Date'] = pd.to_datetime(df_index['Date'], errors='coerce')

In [ ]:
# Count occurrences of each unique value in 'Name_of_Speaker'
name_counts = df_index['Name_of_Speaker'].value_counts()
print(name_counts)


# Plotting the counts
plt.figure(figsize=(10, 8))  # Set the figure size for better readability
name_counts.plot(kind='bar', color='skyblue')  # Create a bar plot
plt.title('Count of Each Speaker in Data')  # Title of the plot
plt.xlabel('Name of Speaker')  # Label for the x-axis
plt.ylabel('Counts')  # Label for the y-axis
plt.xticks(rotation=45, ha='right')  # Rotate the x-axis labels for better readability
plt.tight_layout()  # Adjust subplots to give some padding
plt.show()  # Display the plot

In [ ]:
print(df_index.dtypes)

In [ ]:
df_index

In [ ]:
df_index.reset_index(drop=True, inplace=True)

In [ ]:
df_index.drop(columns=['Unnamed: 0'], inplace=True)

In [ ]:
df_index

# Aggregrating per ecb policy statement

In [ ]:
filtered_df = df_index[df_index['Position'] == 'Monetary Policy Statement']
filtered_df


In [ ]:
# Given dates
dates = [
 '2023-12-14','2023-10-26', '2023-09-14', 
    '2023-07-27', '2023-06-15', '2023-05-04', '2023-03-16', '2023-02-02', 
    '2022-12-15', '2022-10-27', '2022-09-08', '2022-07-21'
]
date_df = pd.DataFrame({'Date': pd.to_datetime(dates)})

# Filter the DataFrame to only include data up to December 2023
df_index = df_index[df_index['Position'] != 'Monetary Policy Statement']


In [ ]:
# Function to find the period for each date in df_index
def find_period(date, periods):
    for i in range(len(periods) - 1):
        if periods[i] <= date < periods[i + 1]:
            return periods[i]
    return periods.iloc[-1]  # Use iloc to access the last element

# Sort periods to ensure correct interval assignment
periods = date_df['Date'].sort_values().reset_index(drop=True)

# Add a period column to the main DataFrame
df_index['Period'] = df_index['Date'].apply(lambda x: find_period(x, periods))

df_index

In [ ]:
# Group by 'Date' and 'Name_of_Speaker', and count the occurrences
speaker_counts = df_index.groupby(['Period', 'Name_of_Speaker']).size().reset_index(name='Counts')

# To display the result
print(speaker_counts)

In [ ]:
import pandas as pd

# Assuming df_index is your DataFrame
# Load your data into df_index, for example by reading from a CSV
# df_index = pd.read_csv('path_to_your_file.csv')

# Step 1: Group by 'Period' and 'Name_of_Speaker', and count the occurrences
speaker_counts = df_index.groupby(['Period', 'Name_of_Speaker']).size().reset_index(name='Counts')

# Step 2: Create a DataFrame with all combinations of 'Period' and 'Name_of_Speaker'
all_periods = pd.DataFrame(df_index['Period'].unique(), columns=['Period'])
all_speakers = pd.DataFrame(df_index['Name_of_Speaker'].unique(), columns=['Name_of_Speaker'])
all_combinations = all_periods.assign(key=1).merge(all_speakers.assign(key=1), on='key').drop('key', axis=1)

# Step 3: Merge and set counts of missing records to zero
full_data = all_combinations.merge(speaker_counts, on=['Period', 'Name_of_Speaker'], how='left')
full_data['Counts'] = full_data['Counts'].fillna(0)  # Set NaN counts to 0


# Filter to find records where counts are less than 5
few_records = full_data[full_data['Counts'] < 5]

# Display results: speakers and periods with counts under 5
print("Speakers and Periods with Counts Under 5:")
print(few_records[['Period', 'Name_of_Speaker']])




In [ ]:
# Count the number of times each speaker appears with counts under 5
speaker_few_count_frequency = few_records.groupby('Name_of_Speaker').size()

# Print the number of times each speaker appears with counts under 5
print("\nFrequency of Speakers with Counts Under 5:")
print(speaker_few_count_frequency)

In [ ]:
import pandas as pd

# Assuming df_index is your DataFrame
# Load your data into df_index, for example by reading from a CSV
# df_index = pd.read_csv('path_to_your_file.csv')

# Calculate mean sentiment per period for each speaker
speaker_result = df_index.groupby(['Period', 'Name_of_Speaker'])['Sentiment'].mean().reset_index()

# Filter out the specific period of 2023-12-14
speaker_result = speaker_result[speaker_result['Period'] != '2023-12-14']

# Display the result
speaker_result



In [ ]:
combined_result=pd.DataFrame(speaker_result)

combined_result

### Verify Data

In [ ]:
# Get unique names of speakers
speaker_result = combined_result['Name_of_Speaker'].unique()

# Convert to DataFrame
unique_speakers_df = pd.DataFrame(combined_result, columns=['Name_of_Speaker'])
unique_speakers_df

In [ ]:
df_governors = pd.read_csv(r"./ECB_Governors_Analysis/ecb_governing_council.csv")

# Overwrite the 'Position' column with 'Executive Role' values where 'Executive Role' is populated
df_governors['Position'] = df_governors['Executive Role'].combine_first(df_governors['Position'])

# Select only the columns 'Full Name', 'Country', and 'Position'
df_governors = df_governors[['Full Name', 'Country', 'Position']]

df_governors

In [ ]:
# Merge the two dataframes on the speaker names to see if all match
merged_df = pd.merge(unique_speakers_df, df_governors, left_on='Name_of_Speaker', right_on='Full Name', how='outer', indicator=True)

# Display mismatches
mismatches = merged_df[merged_df['_merge'] != 'both']

# Check if there are any mismatches
if mismatches.empty:
    print("All speakers in unique_speakers_df match with df_governors.")
else:
    print("There are mismatches between unique_speakers_df and df_governors:")
    print(mismatches[['Name_of_Speaker', 'Full Name', '_merge']])


In [ ]:
# Check vice-versa: Ensure all entries in df_governors are in unique_speakers_df
reverse_merged_df = pd.merge(df_governors, unique_speakers_df, left_on='Full Name', right_on='Name_of_Speaker', how='outer', indicator=True)

# Display reverse mismatches
reverse_mismatches = reverse_merged_df[reverse_merged_df['_merge'] != 'both']

# Check if there are any reverse mismatches
if reverse_mismatches.empty:
    print("All entries in df_governors match with unique_speakers_df and vice-versa.")
else:
    print("There are reverse mismatches between df_governors and unique_speakers_df:")
    print(reverse_mismatches[['Full Name', 'Name_of_Speaker', '_merge']])

## Taking care of Panetta

In [ ]:
combined_result

In [ ]:
# Check if 'Fabio Panetta' is present in the 'Name_of_Speaker' column
fabio_panetta_rows = combined_result[combined_result['Name_of_Speaker'] == 'Fabio Panetta']

# Display the rows with 'Fabio Panetta'
print(fabio_panetta_rows)

In [ ]:

# Function to apply the changes based on conditions
def modify_speaker(row):
    if row['Name_of_Speaker'] == 'Fabio Panetta':
        # Before November 2023
        if row['Period'] < pd.to_datetime('2023-10-31'):
            row['Country'] = 'Fabio Panetta'
        # During November and December 2023
        elif row['Period'] >= pd.to_datetime('2023-11-01'):
            row['Name_of_Speaker'] = 'Fabio Panetta_IT'
            row['Country'] = 'Italy'
    return row

# Apply modifications to the DataFrame
combined_result = combined_result.apply(modify_speaker, axis=1)

# Explicitly set Country for 'Fabio Panetta' before 2023-11
combined_result.loc[(combined_result['Name_of_Speaker'] == 'Fabio Panetta') & (combined_result['Period'] < pd.to_datetime('2023-11-01')), 'Country'] = 'Fabio Panetta'


In [ ]:
# Check if 'Fabio Panetta' is present in the 'Name_of_Speaker' column
fabio_panetta_rows = combined_result[combined_result['Name_of_Speaker'] == 'Fabio Panetta']

# Display the rows with 'Fabio Panetta'
print(fabio_panetta_rows)

In [ ]:
# Display rows where changes are related to Fabio Panetta
filtered_rows = combined_result[combined_result['Name_of_Speaker'].str.contains('Fabio Panetta_IT')]
print(filtered_rows)


In [ ]:
combined_result = combined_result[~((combined_result['Name_of_Speaker'] == 'Ignazio Visco') & (combined_result['Period'] > pd.to_datetime('2023-11-01')))]

# Check if 'Fabio Panetta' is present in the 'Name_of_Speaker' column
fabio_panetta_rows = combined_result[combined_result['Name_of_Speaker'] == 'Ignazio Visco']

# Display the rows with 'Fabio Panetta'
print(fabio_panetta_rows)

In [ ]:
combined_result = combined_result[~((combined_result['Name_of_Speaker'] == 'Piero Cipollone') & (combined_result['Period'] < pd.to_datetime('2023-11-01')))]

In [ ]:
# Check if 'Fabio Panetta' is present in the 'Name_of_Speaker' column
fabio_panetta_rows = combined_result[combined_result['Name_of_Speaker'] == 'Piero Cipollone']

# Display the rows with 'Fabio Panetta'
print(fabio_panetta_rows)

### Now all matched, EDA On Aggregated Speaker

In [ ]:
# Summary statistics
summary_stats = combined_result.describe()
summary_stats

In [ ]:
dict_ECB = {
    'Pablo Hernandez de Cos': 'Spain',
    'Joachim Nagel': 'Germany',
    'Pierre Wunsch': 'Belgium',
    'Yannis Stournaras': 'Greece',
    'Boris Vujcic': 'Croatia',
    'Gaston Reinesch': 'Luxembourg',
    'Francois Villeroy de Galhau': 'France',
    'Robert Holzmann': 'Austria',
    'Peter Kazimir': 'Slovakia',
    'Gediminas Simkus': 'Lithuania',
    'Olli Rehn': 'Finland',
    'Mario Centeno': 'Portugal',
    'Edward Scicluna': 'Malta',
    'Bostjan Vasle': 'Slovenia',
    'Madis Muller': 'Estonia',
    'Martins Kazaks': 'Latvia',
    'Klaas Knot': 'Netherlands',
    'Constantinos Herodotou': 'Cyprus',
    'Gabriel Makhlouf': 'Republic of Ireland',
    'Fabio Panetta_IT': 'Italy',
    'Ignazio Visco': 'Italy'
}


In [ ]:
# Function to update 'Country' based on the 'Name_of_Speaker' using the dictionary
def update_country(row):
    name = row['Name_of_Speaker']
    if name in dict_ECB:
        row['Country'] = dict_ECB[name]
    return row

# Apply the function to the DataFrame
combined_result = combined_result.apply(update_country, axis=1)

In [ ]:
combined_result

In [ ]:
# Check if 'Fabio Panetta' is present in the 'Name_of_Speaker' column
fabio_panetta_rows = combined_result[combined_result['Name_of_Speaker'] == 'Fabio Panetta']

# Display the rows with 'Fabio Panetta'
print(fabio_panetta_rows)

In [ ]:
# Fill NaN values in the 'Country' column with the corresponding 'Name_of_Speaker'
combined_result['Country'] = combined_result['Country'].fillna(combined_result['Name_of_Speaker'])

In [ ]:
combined_result

In [ ]:
# Box plot for sentiment per speaker (now by country)
plt.figure(figsize=(20, 10))
sns.boxplot(x='Country', y='Sentiment', data=combined_result)
plt.title('Sentiment per Speaker (Mapped by Country)')
plt.xticks(rotation=90)
plt.xlabel('Country')
plt.ylabel('Sentiment Per Speaker')
plt.show()

In [ ]:
# Define the categories to filter
categories_to_plot = ['Christine Lagarde', 'Germany', 'France', 'Italy', 'Spain', 'Netherlands']

# Filter the DataFrame to include only these categories
filtered_combined_result = combined_result[combined_result['Country'].isin(categories_to_plot)]

# Box plot for sentiment per speaker (now by selected categories)
plt.figure(figsize=(12, 8))
sns.boxplot(x='Country', y='Sentiment', data=filtered_combined_result)
plt.title('Biggest Eurozone Economies + Lagarde')
plt.xticks(rotation=90)
plt.xlabel('Category')
plt.ylabel('Sentiment Per Speaker')
plt.show()


# Index with Lagarde as Baseline

Sentiment with Lagarde as Baseline

In [ ]:
# Ensure the 'Period' column is in datetime format
combined_result['Period'] = pd.to_datetime(combined_result['Period'])

# Extract Christine Lagarde's sentiment per period
lagarde_sentiment = combined_result[combined_result['Name_of_Speaker'] == 'Christine Lagarde'][['Period', 'Sentiment']]
lagarde_sentiment = lagarde_sentiment.rename(columns={'Sentiment': 'Lagarde_Sentiment'}) 
lagarde_sentiment


In [ ]:
import matplotlib.pyplot as plt

# Plot Christine Lagarde's sentiment over time
plt.figure(figsize=(12, 6))
plt.plot(lagarde_sentiment['Period'], lagarde_sentiment['Lagarde_Sentiment'], marker='o', color='blue')
plt.title("Christine Lagarde's Sentiment Over Time")
plt.xlabel("Period")
plt.ylabel("Lagarde Sentiment")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.show()


In [ ]:
# Merge Lagarde's sentiment with the combined_result dataframe
combined_result_with_lagarde = pd.merge(combined_result, lagarde_sentiment, on='Period', how='left')

# Calculate the absolute difference for each governor's sentiment compared to Lagarde's sentiment
combined_result_with_lagarde['Discordance'] = abs(combined_result_with_lagarde['Sentiment'] - combined_result_with_lagarde['Lagarde_Sentiment'])

# Group by Name_of_Speaker and Period to calculate statistics
discordance_stats = combined_result_with_lagarde.groupby(['Name_of_Speaker', 'Period']).agg({
    'Discordance': ['mean', 'min', 'max', 'std']
}).reset_index()

# Flatten the multi-level columns
discordance_stats.columns = ['Name_of_Speaker', 'Period', 'Mean_Discordance', 'Min_Discordance', 'Max_Discordance', 'Std_Discordance']

discordance_stats

Difference positive or negative

In [ ]:
# Get the list of unique countries excluding Christine Lagarde
countries = combined_result_with_lagarde['Country'].unique()
countries = countries[countries != 'Christine Lagarde']

# Calculate the maximum and minimum differences across all countries to set consistent y-axis limits
min_diff = float('inf')
max_diff = float('-inf')
for country in countries:
    country_data = combined_result_with_lagarde[combined_result_with_lagarde['Country'] == country]
    differences = country_data['Sentiment'] - country_data['Lagarde_Sentiment']
    min_diff = min(min_diff, differences.min())
    max_diff = max(max_diff, differences.max())

# Plot each country's sentiment vs Lagarde's sentiment and the differences
for country in countries:
    country_data = combined_result_with_lagarde[combined_result_with_lagarde['Country'] == country]
    
    # Compute the differences
    country_data['Difference'] = country_data['Sentiment'] - country_data['Lagarde_Sentiment']
    
    # Line plot of differences over time
    plt.figure(figsize=(12, 6))
    plt.plot(country_data['Period'], country_data['Difference'], marker='o', color='blue')
    plt.title(f"Difference in Sentiment between {country} and Christine Lagarde")
    plt.xlabel("Period")
    plt.ylabel("Difference in Sentiment")
    plt.ylim(min_diff, max_diff)  # Set the y-axis limit to ensure consistency
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# Ensure the 'Period' column is in datetime format
combined_result_with_lagarde['Period'] = pd.to_datetime(combined_result_with_lagarde['Period'])

# Compute the differences for each country compared to Christine Lagarde
combined_result_with_lagarde['Difference'] = combined_result_with_lagarde['Sentiment'] - combined_result_with_lagarde['Lagarde_Sentiment']

# Calculate the average difference for each period
average_difference = combined_result_with_lagarde.groupby('Period')['Difference'].mean().reset_index()

# Plot the average difference in sentiment over time
plt.figure(figsize=(12, 6))
plt.plot(average_difference['Period'], average_difference['Difference'], marker='o', color='green')
plt.title("Average Difference in Sentiment Compared to Christine Lagarde")
plt.xlabel("Period")
plt.ylabel("Average Difference in Sentiment")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# List of major Eurozone economies
major_economies = ['France', 'Germany', 'Italy', 'Spain', 'Netherlands']

# Filter the DataFrame to include only the major Eurozone economies
filtered_combined_result = combined_result_with_lagarde[combined_result_with_lagarde['Country'].isin(major_economies)]

# Compute the differences for each country compared to Christine Lagarde
filtered_combined_result['Difference'] = filtered_combined_result['Sentiment'] - filtered_combined_result['Lagarde_Sentiment']

# Calculate the average difference for each period
average_difference = filtered_combined_result.groupby('Period')['Difference'].mean().reset_index()

# Plot the average difference in sentiment over time
plt.figure(figsize=(12, 6))
plt.plot(average_difference['Period'], average_difference['Difference'], marker='o', color='grey')
plt.title("Average Difference in Sentiment Compared to Christine Lagarde (Major Eurozone Economies)")
plt.xlabel("Period")
plt.ylabel("Average Difference in Sentiment")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.show()


# Absolute Differences Vs Largarde

In [ ]:
# Get the list of unique countries excluding Christine Lagarde
countries = combined_result_with_lagarde['Country'].unique()
countries = countries[countries != 'Christine Lagarde']

# Calculate the maximum absolute difference across all countries
max_diff = 0
for country in countries:
    country_data = combined_result_with_lagarde[combined_result_with_lagarde['Country'] == country]
    max_diff = max(max_diff, abs(country_data['Sentiment'] - country_data['Lagarde_Sentiment']).max())

# Plot each country's sentiment vs Lagarde's sentiment and the absolute differences
for country in countries:
    country_data = combined_result_with_lagarde[combined_result_with_lagarde['Country'] == country]
    
    # Compute the absolute differences
    country_data['Absolute_Difference'] = abs(country_data['Sentiment'] - country_data['Lagarde_Sentiment'])
    
    # Line plot of absolute differences over time
    plt.figure(figsize=(12, 6))
    plt.plot(country_data['Period'], country_data['Absolute_Difference'], marker='o', color='red')
    plt.title(f"Absolute Difference in Sentiment between {country} and Christine Lagarde")
    plt.xlabel("Period")
    plt.ylabel("Absolute Difference in Sentiment")
    plt.ylim(0, max_diff)  # Set the y-axis limit to the maximum difference found
    plt.grid(True, linestyle='--', linewidth=0.5)
    plt.xticks(rotation=45)
    plt.show()


In [ ]:
# Ensure the 'Period' column is in datetime format
combined_result_with_lagarde['Period'] = pd.to_datetime(combined_result_with_lagarde['Period'])

# Compute the absolute differences for each country compared to Christine Lagarde
combined_result_with_lagarde['Absolute_Difference'] = abs(combined_result_with_lagarde['Sentiment'] - combined_result_with_lagarde['Lagarde_Sentiment'])

# Calculate the average absolute difference for each period
average_absolute_difference = combined_result_with_lagarde.groupby('Period')['Absolute_Difference'].mean().reset_index()

# Plot the average absolute difference in sentiment over time
plt.figure(figsize=(12, 6))
plt.plot(average_absolute_difference['Period'], average_absolute_difference['Absolute_Difference'], marker='o', color='purple')
plt.title("Average Absolute Difference in Sentiment Compared to Christine Lagarde")
plt.xlabel("Period")
plt.ylabel("Average Absolute Difference in Sentiment")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.show()

In [ ]:
# List of major Eurozone economies
major_economies = ['France', 'Germany', 'Italy', 'Spain', 'Netherlands']

# Filter the DataFrame to include only the major Eurozone economies
filtered_combined_result = combined_result_with_lagarde[combined_result_with_lagarde['Country'].isin(major_economies)]

# Compute the absolute differences for each country compared to Christine Lagarde
filtered_combined_result['Absolute_Difference'] = abs(filtered_combined_result['Sentiment'] - filtered_combined_result['Lagarde_Sentiment'])

# Calculate the average absolute difference for each period
average_absolute_difference = filtered_combined_result.groupby('Period')['Absolute_Difference'].mean().reset_index()

# Plot the average absolute difference in sentiment over time
plt.figure(figsize=(12, 6))
plt.plot(average_absolute_difference['Period'], average_absolute_difference['Absolute_Difference'], marker='o', color='grey')
plt.title("Average Absolute Difference in Sentiment Compared to Christine Lagarde (Major Eurozone Economies)")
plt.xlabel("Period")
plt.ylabel("Average Absolute Difference in Sentiment")
plt.grid(True, linestyle='--', linewidth=0.5)
plt.xticks(rotation=45)
plt.show()

# Combining results with capital weights

In [ ]:
combined_result_with_lagarde

In [ ]:
combined_result_with_lagarde.isnull().sum()

In [ ]:
speakers_to_remove = ["Boris Vujcic", "Edward Scicluna", "Gaston Reinesch"]

# Filter the DataFrame to exclude the listed speakers
df_to_merge = combined_result_with_lagarde[~combined_result_with_lagarde['Name_of_Speaker'].isin(speakers_to_remove)]


In [ ]:
import os
os.chdir('/Users/ruimaciel/Desktop/Barcelona/Master_Thesis/ECB_Perceived_Cacophony')

current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)


In [ ]:
capital_key = pd.read_csv(r"./Rui_final_notebooks/capital_key_ecb.csv")
capital_key.columns

In [ ]:
capital_key.drop(columns=['Paid-up Capital (€)'], inplace=True)

In [ ]:
capital_key.rename(columns={'Capital Key %': 'Capital Key'}, inplace=True)

In [ ]:
capital_key['Country'] = capital_key['Country'].replace({'The Netherlands': 'Netherlands', 'Ireland': 'Republic of Ireland'})

In [ ]:
capital_key

In [ ]:
df_merged = df_to_merge.merge(capital_key[['Country', 'Capital Key']], on='Country', how='left')
df_merged

In [ ]:
sum_unique_capital_keys = df_merged['Capital Key'].drop_duplicates().sum()

# Display the sum of unique 'Capital Key' values
print(sum_unique_capital_keys)

In [ ]:
remainder = 100 - sum_unique_capital_keys
value_per_person = remainder / 5
print(value_per_person)

# List of specific speakers to update
speakers_to_update = ["Fabio Panetta", "Philip Lane", "Isabel Schnabel", "Frank Elderson", "Luis de Guindos"]

# Update 'Capital Key' for the specified speakers
df_merged.loc[df_merged['Name_of_Speaker'].isin(speakers_to_update), 'Capital Key'] = value_per_person

# Update 'Capital Key' for the specified speakers
df_merged.loc[df_merged['Name_of_Speaker'].isin(speakers_to_update), 'Capital Key'] = value_per_person


In [ ]:
# Counting unique entries for each speaker in the column 'Name_of_Speaker'
unique_counts = df_merged['Name_of_Speaker'].value_counts()

# Display the counts
print(unique_counts)

In [ ]:
# List of speakers to check
speakers_to_check = ["Gabriel Makhlouf", "Constantinos Herodotou", "Frank Elderson"]

# Find all unique periods in the DataFrame
all_periods = df_merged['Period'].unique()

# Dictionary to hold the missing periods for each speaker
missing_periods = {}

# Check each speaker for missing periods
for speaker in speakers_to_check:
    # Filter the DataFrame for the current speaker and get their unique periods
    speaker_periods = df_merged[df_merged['Name_of_Speaker'] == speaker]['Period'].unique()
    
    # Find the set difference: periods that are missing for the speaker
    missing_periods[speaker] = list(set(all_periods) - set(speaker_periods))

# Output the missing periods for each speaker
for speaker, periods in missing_periods.items():
    print(f"Missing periods for {speaker}: {periods}")

In [ ]:
import pandas as pd
import pandas as pd

# Define the speakers and their missing periods
missing_data = {
    "Gabriel Makhlouf": pd.Timestamp('2023-07-27 00:00:00'),
    "Constantinos Herodotou": pd.Timestamp('2022-07-21 00:00:00'),
    "Frank Elderson": pd.Timestamp('2022-07-21 00:00:00')
}

# Create new rows for each speaker and their missing period
new_rows = []
for speaker, missing_period in missing_data.items():
    # Filter the DataFrame for the current speaker and periods before the missing one
    speaker_df = df_merged[(df_merged['Name_of_Speaker'] == speaker) & (df_merged['Period'] < missing_period)]
    closest_row = None

    # Check if there are entries before the missing period
    if not speaker_df.empty:
        closest_index = (speaker_df['Period'] - missing_period).abs().idxmin()
        closest_row = speaker_df.loc[closest_index]
    else:
        # If no earlier periods exist, look for the next closest period after the missing one
        speaker_df = df_merged[(df_merged['Name_of_Speaker'] == speaker) & (df_merged['Period'] > missing_period)]
        if not speaker_df.empty:
            closest_index = (speaker_df['Period'] - missing_period).abs().idxmin()
            closest_row = speaker_df.loc[closest_index]
        else:
            print(f"No periods found for {speaker}")

    if closest_row is not None:
        # Copy the closest row and modify it
        base_row = closest_row.copy()
        base_row['Period'] = missing_period

        # Append the modified row to new_rows list
        new_rows.append(base_row)

# Convert new_rows list to a DataFrame
new_rows_df = pd.DataFrame(new_rows)

# Append these new rows to the original DataFrame
df_merged = pd.concat([df_merged, new_rows_df], ignore_index=True)

# Verify by displaying the newly added rows
print(df_merged.tail(len(new_rows)))  # Show the last few rows to verify new entries



In [ ]:
# Multiply every value in the 'Capital Key' column by 0.01
df_merged['Capital Key'] = df_merged['Capital Key'] * 0.01

# Display the updated DataFrame to verify the changes
print(df_merged[['Name_of_Speaker', 'Capital Key']].head())  # Show the first few rows as a sample

In [ ]:
df_merged

# Going for index

In [ ]:
df_filtered = df_merged.dropna(subset=['Capital Key'])

# Define a function to calculate the weighted average for a group
def weighted_average(group):
    if group['Capital Key'].sum() > 0:
        return (group['Absolute_Difference'] * group['Capital Key']).sum() / group['Capital Key'].sum()
    else:
        return None  # Handle cases where the sum of weights is zero

# Calculate the weighted average of 'Absolute_Difference' by 'Period' using 'Capital Key' as weights
weighted_averages = df_filtered.groupby('Period').apply(weighted_average)

# Plot the weighted averages
plt.figure(figsize=(10, 6))  # Set the figure size
plt.plot(weighted_averages.index, weighted_averages, marker='o', linestyle='-', color='b')  # Line plot with markers
plt.title('Weighted Average of Absolute Difference by Period')  # Title of the plot
plt.xlabel('Period')  # X-axis label
plt.ylabel('Weighted Average')  # Y-axis label
plt.grid(True)  # Enable grid for better readability
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()  # Display the plot

In [ ]:
# Exclude rows where 'Capital Key' is NaN
df_filtered = df_merged.dropna(subset=['Capital Key'])

# Define a function to calculate the weighted average for 'Absolute_Difference'
def weighted_average_abs_diff(group):
    if group['Capital Key'].sum() > 0:
        return (group['Absolute_Difference'] * group['Capital Key']).sum() / group['Capital Key'].sum()
    else:
        return None  # Handle cases where the sum of weights is zero

# Calculate the weighted average of 'Absolute_Difference' by 'Period' using 'Capital Key' as weights
weighted_averages_abs_diff = df_filtered.groupby('Period').apply(weighted_average_abs_diff)

# Define a function to calculate the weighted average for 'Difference'
def weighted_average_diff(group):
    if group['Capital Key'].sum() > 0:
        return (group['Difference'] * group['Capital Key']).sum() / group['Capital Key'].sum()
    else:
        return None  # Handle cases where the sum of weights is zero

# Calculate the weighted average of 'Difference' by 'Period' using 'Capital Key' as weights
weighted_averages_diff = df_filtered.groupby('Period').apply(weighted_average_diff)

# Plot the weighted averages on the same graph
plt.figure(figsize=(12, 8))  # Set the figure size
plt.plot(weighted_averages_abs_diff.index, weighted_averages_abs_diff, marker='o', linestyle='-', color='b', label='Absolute Difference')  # Line plot with markers for Absolute Difference
plt.plot(weighted_averages_diff.index, weighted_averages_diff, marker='x', linestyle='--', color='r', label='Difference')  # Line plot with markers for Difference
plt.title('Weighted Averages of Absolute Difference and Difference by Period')  # Title of the plot
plt.xlabel('Period')  # X-axis label
plt.ylabel('Weighted Average')  # Y-axis label
plt.grid(True)  # Enable grid for better readability
plt.xticks(rotation=45)  # Rotate x-axis labels for better visibility
plt.legend()  # Add a legend to distinguish the lines
plt.tight_layout()  # Adjust layout to prevent clipping of tick-labels
plt.show()  # Display the plot



In [ ]:
import pandas as pd
import numpy as np

# Assuming df_merged is already defined and contains your data
# Convert 'Period' to date-only format if it includes time
df_merged['Period'] = pd.to_datetime(df_merged['Period']).dt.date

# Create a pivot table with countries on rows and periods as columns for 'Absolute_Difference'
pivot_table_ad = df_merged.pivot_table(index='Country', columns='Period', values='Absolute_Difference', aggfunc='mean')

# Since 'Capital Key' should be constant for each country, take the first non-null value for each country
capital_keys = df_merged.dropna(subset=['Capital Key']).groupby('Country')['Capital Key'].first()

# Join the 'Capital Key' to the pivot table
final_matrix = pivot_table_ad.join(capital_keys)

# Calculate the weighted index across all countries for each period
weighted_index = df_merged.dropna(subset=['Capital Key']).groupby('Period').apply(lambda x: (x['Absolute_Difference'] * x['Capital Key']).sum() / x['Capital Key'].sum())

# Expand weighted_index into a DataFrame to match the structure of final_matrix
# This will add 'Weighted Index' directly as a row into the DataFrame
weighted_index_df = pd.DataFrame(weighted_index).T  # Transpose to make it a row
weighted_index_df.index = ['Weighted Index']  # Name the index
weighted_index_df['Capital Key'] = None # Add the 'Capital Key' column with None as value

# Concatenate weighted_index_df at the bottom of final_matrix
final_matrix = pd.concat([final_matrix, weighted_index_df])

# Drop 'Christine Lagarde' if present in the Country index
if 'Christine Lagarde' in final_matrix.index:
    final_matrix = final_matrix.drop('Christine Lagarde', axis=0)

    # Sort final_matrix by 'Capital Key'
final_matrix = final_matrix.sort_values(by='Capital Key', ascending=False)

# Set display format for numerical values to three decimal places
pd.set_option('display.float_format', '{:.3f}'.format)

# Print and display results
print("Pivot Table of Absolute Differences:")
final_matrix


In [ ]:
import pandas as pd
import numpy as np

# Assuming df_merged is already defined and contains your data
# Convert 'Period' to date-only format if it includes time
df_merged['Period'] = pd.to_datetime(df_merged['Period']).dt.date

# Create a pivot table with countries on rows and periods as columns for 'Absolute_Difference'
pivot_table_ad = df_merged.pivot_table(index='Country', columns='Period', values='Difference', aggfunc='mean')

# Since 'Capital Key' should be constant for each country, take the first non-null value for each country
capital_keys = df_merged.dropna(subset=['Capital Key']).groupby('Country')['Capital Key'].first()

# Join the 'Capital Key' to the pivot table
final_matrix = pivot_table_ad.join(capital_keys)

# Calculate the weighted index across all countries for each period
weighted_index = df_merged.dropna(subset=['Capital Key']).groupby('Period').apply(lambda x: (x['Difference'] * x['Capital Key']).sum() / x['Capital Key'].sum())

# Expand weighted_index into a DataFrame to match the structure of final_matrix
# This will add 'Weighted Index' directly as a row into the DataFrame
weighted_index_df = pd.DataFrame(weighted_index).T  # Transpose to make it a row
weighted_index_df.index = ['Weighted Index']  # Name the index
weighted_index_df['Capital Key'] = None  # Add the 'Capital Key' column with None as value

# Concatenate weighted_index_df at the bottom of final_matrix
final_matrix = pd.concat([final_matrix, weighted_index_df])

# Drop 'Christine Lagarde' if present in the Country index
if 'Christine Lagarde' in final_matrix.index:
    final_matrix = final_matrix.drop('Christine Lagarde', axis=0)

# Sort final_matrix by 'Capital Key'
final_matrix = final_matrix.sort_values(by='Capital Key', ascending=False)

# Set display format for numerical values to three decimal places
pd.set_option('display.float_format', '{:.3f}'.format)

# Print and display results
print("Pivot Table of Differences:")
final_matrix

In [ ]:

# Assuming df_merged is already defined and contains your data
# Filter the DataFrame for entries related to 'Christine Lagarde'
df_christine = df_merged[df_merged['Country'] == 'Christine Lagarde']

# Convert 'Period' to date-only format if it includes time
df_christine['Period'] = pd.to_datetime(df_christine['Period']).dt.date

# Create a pivot table with 'Period' as columns and values as 'Sentiment'
# This assumes 'Sentiment' is a column in df_merged that you want to analyze
pivot_table_christine = df_christine.pivot_table(index='Country', columns='Period', values='Sentiment', aggfunc='mean')

pd.set_option('display.float_format', '{:.3f}'.format)

# Print and display results
print("Sentiment Over Time for Christine Lagarde:")
pivot_table_christine

# Attempt to regress

In [ ]:
data = {
    '2022-07-21': [0.50],
    '2022-09-08': [0.75],
    '2022-10-27': [0.75],
    '2022-12-15': [0.50],
    '2023-02-02': [0.50],
    '2023-03-16': [0.50],
    '2023-05-04': [0.25],
    '2023-06-15': [0.25],
    '2023-07-27': [0.25],
    '2023-09-14': [0.25],
    '2023-10-26': [0.00]
}

# Create a DataFrame with the dictionary where keys are now the column headers
df_interests_transposed = pd.DataFrame(data, index=['Interest Rate Raise'])

# Display the DataFrame
df_interests_transposed

In [ ]:
df_filtered

In [ ]:
# Assuming df_merged is already defined and contains your data
# Convert 'Period' to date-only format if it includes time
df_merged['Period'] = pd.to_datetime(df_merged['Period']).dt.date

# Create a pivot table with countries on rows and periods as columns for 'Lagarde_Sentiment'
# Using 'mean' as an aggregation function, or choose another suitable one
pivot_table_ad = df_merged.pivot_table(index='Country', columns='Period', values='Lagarde_Sentiment', aggfunc='mean')

# Filter the pivot table to show only the row for Christine Lagarde
pivot_table_lagarde = pivot_table_ad.loc[pivot_table_ad.index == 'Christine Lagarde']

# Print the final matrix for Christine Lagarde
print("Sentiment Over Time for Christine Lagarde:")
print(pivot_table_lagarde)


In [ ]:
# Assuming df_merged is already defined and contains your data
# Convert 'Period' to date-only format if it includes time
df_merged['Period'] = pd.to_datetime(df_merged['Period']).dt.date

# Create a pivot table with countries on rows and periods as columns for 'Lagarde_Sentiment'
pivot_table_ad = df_merged.pivot_table(index='Country', columns='Period', values='Lagarde_Sentiment', aggfunc='mean')

# Filter the pivot table to show only the row for Christine Lagarde
pivot_table_lagarde = pivot_table_ad.loc[pivot_table_ad.index == 'Christine Lagarde']

# Prepare the interest rate raise data
data = {
    '2022-07-21': [0.50],
    '2022-09-08': [0.75],
    '2022-10-27': [0.75],
    '2022-12-15': [0.50],
    '2023-02-02': [0.50],
    '2023-03-16': [0.50],
    '2023-05-04': [0.25],
    '2023-06-15': [0.25],
    '2023-07-27': [0.25],
    '2023-09-14': [0.25],
    '2023-10-26': [0.00]
}

# Create a DataFrame with the dictionary where keys are now the column headers
df_interests_transposed = pd.DataFrame(data, index=['Interest Rate Raise'])

# Convert date strings to datetime format in df_interests_transposed to match pivot_table_lagarde
df_interests_transposed.columns = pd.to_datetime(df_interests_transposed.columns)

# Align the date formats in pivot_table_lagarde
pivot_table_lagarde.columns = pd.to_datetime(pivot_table_lagarde.columns)

# Drop columns with NaNs in either DataFrame
common_columns = pivot_table_lagarde.columns.intersection(df_interests_transposed.columns)
pivot_table_lagarde = pivot_table_lagarde[common_columns].dropna(axis=1, how='any')
df_interests_transposed = df_interests_transposed[common_columns].dropna(axis=1, how='any')

# Merge (concatenate) the sentiment data with the interest rate data
combined_matrix = pd.concat([pivot_table_lagarde, df_interests_transposed])

# Print the combined matrix for Christine Lagarde and Interest Rate Raise
print("Combined Matrix for Christine Lagarde and Interest Rate Raise:")
combined_matrix

In [ ]:
import pandas as pd
import statsmodels.api as sm

# Ensure the index is reset to have a proper DataFrame structure
combined_matrix = combined_matrix.T  # Transpose to have dates as index

# The combined matrix has more than just two columns after transpose
# To correctly assign names, let's inspect the transposed DataFrame first
print(combined_matrix.head())

# The correct columns assignment
combined_matrix.columns = combined_matrix.iloc[0]  # Assign the first row as header

# Now the DataFrame should have the correct columns
print(combined_matrix.head())

# Ensure columns are correctly formatted
combined_matrix.columns = ['Christine Lagarde Sentiment', 'Interest Rate Raise']

# Define the dependent variable (Y) and independent variable (X)
Y = combined_matrix['Interest Rate Raise']
X = combined_matrix['Christine Lagarde Sentiment']

# Add a constant to the independent variable (X) for the OLS regression
X = sm.add_constant(X)

# Perform the OLS regression with robust standard errors
model = sm.OLS(Y, X).fit(cov_type='HC3')

# Print the summary of the regression with robust standard errors
print(model.summary())


In [ ]:
import pandas as pd
import statsmodels.api as sm

# Assuming combined_matrix is already defined as per the previous code

# Ensure the index is reset to have a proper DataFrame structure
combined_matrix = combined_matrix.T  # Transpose to have dates as index

# The combined matrix has more than just two columns after transpose
# To correctly assign names, let's inspect the transposed DataFrame first
print(combined_matrix.head())

# The correct columns assignment
combined_matrix.columns = combined_matrix.iloc[0]  # Assign the first row as header

# Now the DataFrame should have the correct columns
print(combined_matrix.head())

# Ensure columns are correctly formatted
combined_matrix.columns = ['Christine Lagarde Sentiment', 'Interest Rate Raise']

# Shift the independent variable (Christine Lagarde Sentiment) by one period
combined_matrix['Christine Lagarde Sentiment Lagged'] = combined_matrix['Christine Lagarde Sentiment'].shift(1)

# Drop the first row since it will have a NaN value after the shift
combined_matrix = combined_matrix.dropna()

# Define the dependent variable (Y) and independent variable (X)
Y = combined_matrix['Interest Rate Raise']
X = combined_matrix['Christine Lagarde Sentiment Lagged']

# Add a constant to the independent variable (X) for the OLS regression
X = sm.add_constant(X)

# Perform the OLS regression with robust standard errors
model = sm.OLS(Y, X).fit(cov_type='HC3')

# Print the summary of the regression with robust standard errors
print(model.summary())


# Attempt at finding the drivers of the cacophony

In [ ]:
import pandas as pd

# Assuming df_merged is already defined and contains your data
# Convert 'Period' to date-only format if it includes time
df_merged['Period'] = pd.to_datetime(df_merged['Period']).dt.date

# Create a pivot table with countries on rows and periods as columns for 'Sentiment'
pivot_table_lagarde = df_merged.pivot_table(index='Country', columns='Period', values='Sentiment', aggfunc='mean')

# Prepare the interest rate raise data
data = {
    '2022-07-21': [0.50],
    '2022-09-08': [0.75],
    '2022-10-27': [0.75],
    '2022-12-15': [0.50],
    '2023-02-02': [0.50],
    '2023-03-16': [0.50],
    '2023-05-04': [0.25],
    '2023-06-15': [0.25],
    '2023-07-27': [0.25],
    '2023-09-14': [0.25],
    '2023-10-26': [0.00]
}

# Create a DataFrame with the dictionary where keys are now the column headers
df_interests_transposed = pd.DataFrame(data, index=['Interest Rate Raise'])

# Convert date strings to datetime format in df_interests_transposed to match pivot_table_lagarde
df_interests_transposed.columns = pd.to_datetime(df_interests_transposed.columns)

# Align the date formats in pivot_table_lagarde
pivot_table_lagarde.columns = pd.to_datetime(pivot_table_lagarde.columns)

# Ensure that all columns are included, filling NaNs where necessary
common_columns = pivot_table_lagarde.columns.union(df_interests_transposed.columns)
pivot_table_lagarde = pivot_table_lagarde.reindex(columns=common_columns)
df_interests_transposed = df_interests_transposed.reindex(columns=common_columns)

# Merge (concatenate) the sentiment data with the interest rate data
combined_matrix = pd.concat([pivot_table_lagarde, df_interests_transposed])

# Print the combined matrix for Christine Lagarde and Interest Rate Raise
print("Combined Matrix for Countries and Interest Rate Raise:")
combined_matrix


In [ ]:
import statsmodels.api as sm

# Select only the specified countries
countries_to_keep = ['Fabio Panetta', 'Frank Elderson', 'Isabel Schnabel', 'Luis de Guindos', 'Philip Lane']
X = combined_matrix.drop(index='Interest Rate Raise').drop(index='Christine Lagarde').T[countries_to_keep]

# The Y variable (Interest Rate Raise)
Y = combined_matrix.loc['Interest Rate Raise'].T

# Add a constant to the independent variables matrix (X)
X = sm.add_constant(X)

# Fit the OLS model
model = sm.OLS(Y, X, missing='drop').fit()

# Print the summary of the regression
print(model.summary())
